# Baseline Model

Below we have the baseline model using an SVM network.

In [30]:
"""
Baseline Model - SVM
"""

import cv2
import numpy as np
import os

from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

np.random.seed(0)
RANDOM_NUM = np.random.randint(1000)

In [31]:
DATA_PATH = "KLARR_NET_data/fer_2013"

IMG_WIDTH = 48
IMG_HEIGHT = 48

def create_dataset(img_folder):
    """ Creates a dataset of the images in the given folder and returns the
    data and labels array.
    """
    img_data_array = []
    class_name = []
   
    for class_num, dir in enumerate(os.listdir(img_folder)):
        print(f"Class Num: {class_num}, Directory: {dir}")

        for i, filename in enumerate(os.listdir(os.path.join(img_folder, dir))):
            if i % 1000 == 0:
                print(i)

            # Read the image in grayscale
            image_path = os.path.join(img_folder, dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(
                image,
                (IMG_HEIGHT, IMG_WIDTH),
                interpolation=cv2.INTER_AREA
            )

            # Convert to numpy and normalize between 0 and 1
            image = np.array(image).astype('float32').flatten() / 255

            img_data_array.append(image)
            class_name.append(class_num)

    return np.array(img_data_array), np.array(class_name)

X_train, y_train = create_dataset(DATA_PATH)
X_train, X_test, y_train, y_test = train_test_split(
    X_train,
    y_train,
    train_size=0.7,
    random_state=RANDOM_NUM)

Class Num: 0, Directory: angry
0
1000
2000
3000
4000
Class Num: 1, Directory: fear
0
1000
2000
3000
4000
5000
Class Num: 2, Directory: happy
0
1000
2000
3000
4000
5000
Class Num: 3, Directory: neutral
0
1000
2000
3000
4000
5000
6000
Class Num: 4, Directory: sad
0
1000
2000
3000
4000
5000
6000
Class Num: 5, Directory: surprise
0
1000
2000
3000


In [32]:
CLASSES = ['angry', 'fear', 'happy' ,'neutral' ,'sad' ,'surprise']

def print_class_distribution_in_dataset(dataset_labels):
    num_classes = [0] * len(CLASSES)
    for label in dataset_labels:
        num_classes[label] += 1
    
    for i, num_of_class in enumerate(num_classes):
        print(f"Class: {CLASSES[i]}, Number of Classes: {num_of_class}")

print(f"Baseline Model Training Set Size: {len(X_train)}")
print_class_distribution_in_dataset(y_train)

print(f"\nBaseline Model Test Set Size: {len(X_test)}")
print_class_distribution_in_dataset(y_test)


Baseline Model Training Set Size: 22408
Class: angry, Number of Classes: 3467
Class: fear, Number of Classes: 3536
Class: happy, Number of Classes: 3958
Class: neutral, Number of Classes: 4365
Class: sad, Number of Classes: 4289
Class: surprise, Number of Classes: 2793

Baseline Model Test Set Size: 9604
Class: angry, Number of Classes: 1450
Class: fear, Number of Classes: 1545
Class: happy, Number of Classes: 1827
Class: neutral, Number of Classes: 1821
Class: sad, Number of Classes: 1773
Class: surprise, Number of Classes: 1188


In [33]:
# Instantiate the SVM classifier
# svm_classifier = svm.SVC(gamma=0.001)
svm_classifier = svm.SVC(
    kernel='linear',
    C=1,
    decision_function_shape='ovo',
)

# Fit to the training data
svm_classifier.fit(X_train, y_train)

SVC(C=1, decision_function_shape='ovo', kernel='linear')

In [36]:
# y_pred = svm_classifier.predict(X_test)
# classification_report = metrics.classification_report(y_test, y_pred)
# confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

# baseline_test_acc = svm_classifier.score(X_test, y_test)
print(f"Baseline Model Test Accuracy: {baseline_test_acc}")

print(f"Classification report for classifier {svm_classifier}:")
print(f"\n{classification_report}")

print(f"\nConfusion matrix:\n\n{confusion_matrix}")

Baseline Model Test Accuracy: 0.3263223656809663
Classification report for classifier SVC(C=1, decision_function_shape='ovo', kernel='linear'):

              precision    recall  f1-score   support

           0       0.23      0.25      0.24      1450
           1       0.25      0.24      0.24      1545
           2       0.40      0.39      0.40      1827
           3       0.33      0.36      0.35      1821
           4       0.29      0.29      0.29      1773
           5       0.49      0.43      0.46      1188

    accuracy                           0.33      9604
   macro avg       0.33      0.33      0.33      9604
weighted avg       0.33      0.33      0.33      9604


Confusion matrix:

[[359 217 250 276 266  82]
 [229 368 212 280 282 174]
 [279 217 720 262 264  85]
 [247 248 223 651 343 109]
 [321 239 268 334 522  89]
 [127 176 112 141 118 514]]


In [40]:
# # Save model parameters so we can load it later
# from joblib import dump, load
# from pathlib import Path

# BASELINE_MODEL_LOCATION = "models/baseline_svm_1.joblib"

# Path("models").mkdir(parents=True, exist_ok=True)
# dump(svm_classifier, BASELINE_MODEL_LOCATION)

# # To load the model, uncomment this line
# clf = load(BASELINE_MODEL_LOCATION)

['models/baseline_svm_1.joblib']

# CNN Implementation

In [ ]:
"""
CNN model Implementation
"""

import os 
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data

TRAIN_PATH = ""
TEST_PATH =  ""
VALIDATION_PATH =  ""

TRANSFORM_IMG = transforms.Compose([
    #transforms.Grayscale(),
    transforms.ToTensor(),
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_PATH, transform=TRANSFORM_IMG)
print('Number of Train Images : ', len(train_data))

test_data = torchvision.datasets.ImageFolder(root=TEST_PATH, transform=TRANSFORM_IMG)
print('Number of Test Images : ', len(test_data))

validation_data = torchvision.datasets.ImageFolder(root=VALIDATION_PATH, transform=TRANSFORM_IMG)
print('Number of Validation Images : ', len(validation_data))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim #for gradient descent

class KLARR_NET(nn.Module):
  def __init__(self):
        torch.manual_seed(1000)
        super(ASLClassifer, self).__init__()
        self.name = 'ASLClassifier'
        self.conv1 = nn.Conv2d(3, 5, 5) # TODO : make sure this input size fits with the VGG16 feature size
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 5)
        self.conv3 = nn.Conv2d(10, 10, 5)
        self.fc1 = nn.Linear(10*24*24, 72)
        self.fc2 = nn.Linear(72, 9)

  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = self.pool(F.relu(self.conv3(x)))
      x = x.view(-1, 10*24*24)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)

      x = x.squeeze(1) # Flatten to [batch_size]

      return x

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    for img, label in iter(data_loader):
      out = model(img)
      pred = out.max(1, keepdim=True)[1]
      correct += pred.eq(label.view_as(pred)).sum().item()
      total += img.shape[0]
    #print('Accuracy :', correct/total)
    return correct / total

def train(model, train_data, val_data, batch_size, learning_rate, iterations):

  # torch.manual_seed(1000)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  iters, losses, train_acc, val_acc = [], [], [], []
  val_loader = data.DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=0)
  train_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)

  n = 0
  for iteraion in range(interations):
    print('ITERATION : ', epoch)
    for img, label in iter(train_loader):
      out = model(img)

      loss = criterion(out, label)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    iters.append(n)
    losses.append(float(loss) / batch_size)
    train_acc.append(get_accuracy(model, train_loader))
    val_acc.append(get_accuracy(model, val_loader))
    n += 1

    # Save the current model (checkpoint) to a file
    # model_path = get_model_name(model.name, batch_size, learning_rate, epoch)
    # torch.save(model.state_dict(), model_path)
    
  # Plotting
  plt.title("Training Curve")
  plt.plot(iters, losses, label="Train")
  plt.xlabel("Iterations")
  plt.ylabel("Loss")
  plt.show()

  plt.title("Training Curve")
  plt.plot(iters, train_acc, label="Train")
  plt.plot(iters, val_acc, label="Validation")
  plt.xlabel("Iterations")
  plt.ylabel("Training Accuracy")
  plt.legend(loc='best')
  plt.show()

  print("Final Training Accuracy: {}".format(train_acc[-1]))
  print("Final Validation Accuracy: {}".format(val_acc[-1]))

In [ ]:
import torchvision.models
alexnet = torchvision.models.vgg16(pretrained=True)

In [ ]:
Train_PATH = ""
validation_PATH = ""
Test_PATH = ""

classes= ['angry','fear','happy','neutral','sad','surprise']

train_data_loader = data.DataLoader(train_data, batch_size=1, shuffle=False,  num_workers=0)

test_data_loader = data.DataLoader(test_data, batch_size=1, shuffle=False,  num_workers=0)

validation_data_loader = data.DataLoader(validation_data, batch_size=1, shuffle=False,  num_workers=0)

In [ ]:
"""
Code to import the vgg16 features of our images into separate image folders in google drive
"""

def save_tensor(feature, folder_name, i):
  if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
  filename = folder_name + '/' + str(i) + '.tensor'
  if not os.path.isfile(filename):
    torch.save(feature.squeeze(0), folder_name + '/' + str(i) + '.tensor')


iteration_no = 0 
for img, label in train_data_loader:
  features = alexnet.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())
  folder_name = Train_PATH  + str(classes[label])

  save_tensor(features_tensor, folder_name, iteration_no)
  iteration_no += 1

iteration_no = 0 
for img, label in test_data_loader:
  features = alexnet.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())
  folder_name = Test_PATH + str(classes[label])
  
  save_tensor(features_tensor, folder_name, iteration_no)
  iteration_no += 1

iteration_no = 0
for img, label in validation_data_loader:
  features = alexnet.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())
  folder_name = validation_PATH  + str(classes[label])

  save_tensor(features_tensor, folder_name, iteration_no)
  iteration_no += 1

In [ ]:
# Import features from location on Google Drive
feature_train_set = torchvision.datasets.DatasetFolder(Train_PATH, loader=torch.load, extensions=('.tensor'))
feature_val_set = torchvision.datasets.DatasetFolder(validation_PATH, loader=torch.load, extensions=('.tensor'))
feature_test_set = torchvision.datasets.DatasetFolder(Test_PATH, loader=torch.load, extensions=('.tensor'))